In [1]:
from franc_lib.semantics import Semantics
from franc_lib.lexical import Preprocessing

/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
s = Semantics()

In [3]:
s.sentiment_analysis(text="Essa geladeira é ruim")

Trainning....
[LibLinear]

/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/franciscone/.envs/nlp/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 3.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Finished !
Precisão 55.5204
[[  20   16    3   12   60   38]
 [   1  105   46   66  125  123]
 [   0   47   58  128  240   79]
 [   4   27   25  281 1191  271]
 [   0    9    5  151 3219 1656]
 [   2    5    4   25 1547 3689]]


array([2.])

In [4]:
s.get_confusion_matrix()
# s.classifier.predict(s.X_test)

v/v	 0.0	 1.0	 2.0	 3.0	 4.0	 5.0
0.0	 20	 16	 3	 12	 60	 38	
1.0	 1	 105	 46	 66	 125	 123	
2.0	 0	 47	 58	 128	 240	 79	
3.0	 4	 27	 25	 281	 1191	 271	
4.0	 0	 9	 5	 151	 3219	 1656	
5.0	 2	 5	 4	 25	 1547	 3689	


In [ ]:
accuracy_score(test_classes, mlp.predict(X_test))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=100,n_jobs=3, criterion="entropy", verbose=1)
rf.fit(X, train_classes)

In [ ]:
accuracy_score(test_classes, rf.predict(X_test))

In [ ]:
sentence= ""
print(preprocessing(sentence))
X = transformer.transform([preprocessing(sentence)])
rf.predict(X)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(15, weights='distance')
knn.fit(X, train_classes)

In [ ]:
accuracy_score(test_classes, knn.predict(X_test))